# HW3 - Group 31

## Libraries

### 1.1) Get the list of movies

In this step we get the list of all movies urls in the movies html files. Each member of the group used this code to get the list of movies and then html files.

In [ ]:
def get_movieList (path) :
    movies = pd.DataFrame(pd.read_html(path + "\\movies1.html")[0]) #put the content of html file in a dataframe and get the first column
    movies.drop('Id', inplace=True, axis = 1)
    return movies
path = os.getcwd() #The address of directory where Movies.html files exist
movies = get_movieList(path)  #this function will give us list of movies urls in the html file of movies which exist in the path address   

### 1.2) Crawl Wikipedia

Now, we crawl each wikipedia page to get html files

In [ ]:
def save_html(movies) :
    for i in range(len(movies)):
        try:
            response = rq.get(movies.URL[i])
        except rq.exceptions.RequestException as e: #if we got blocked by wiki we apply a time sleep
            print(e)
            time.sleep(20*60 + 30)
            response = rq.get(movies.URL[i])
        soup = BeautifulSoup(response.text, 'html.parser')
        f = open('article_'+str(i)+'.html','w')
        f.write(str(soup))
        f.close()
        time.sleep(random.choice(range(1,6)) #time sleep between each request
                   
save_html(movies)

### 1.3) Parse downloaded pages

In this step, we should parse HTML pages, get the specefic information we want and then save it as TSV files

## 2) search engine

###  preprocessing

#### All the TSV Files were preprocessed by :
1) Tokenization

2) Removing stop words

3) Removing punctuation

4) Stemming

5) Removing [] , ""


In [ ]:

def clean(text):
    stop_words = set(stopwords.words('english')) 
    stemmer = PorterStemmer()
    text = text.lower()
    words = word_tokenize(text) #devide the text into substrings
    filtered1 = [w for w in words if not w in stop_words] #remove stop words
    filtered2 = list(filter(lambda word: word not in string.punctuation, filtered1))
    filtered3 = []
    for word in filtered2:
        try:
            filtered3 += re.findall(r'\w+', word) 
        except:
            pass
    
    filtered3 = [stemmer.stem(w) for w in filtered3] #stemming
    filtered4 = [c.replace("''", "").replace("``", "") for c in filtered3 ] #removing useless '' and  `` characters
    filtered4 = [f for f in filtered4 if len(f)>1]
    return filtered4

### 2.1) Conjunctive query

#### 2.1.1) creating index

In [ ]:
# this function save an object to desired path as a json file
def savetojson(pathfile, obj):
    with open(pathfile, "w" ,encoding="utf-8") as out_file:
        out_file.write(json.dumps(obj, ensure_ascii = False))
        out_file.close()
        
        
#this function gives us 3 json files, first the vocabulary which contains a dictionary of term ids and words
#second the tsvs which containd a dictionary of cleared text of intro and plot for each document
#third docwords which contains a dictionary of tokens of each document
def get_vocab_index(path) :
    allwords = list()
    docwords = dict() # point each document to its containing words
    tsvs = dict()
    vocabulary = dict() # point each term id to a word
    for i in range(0,30000):
        with open(path+"\\TSV\\article_" + str(i) + ".tsv", encoding = "utf-8") as fd:
            rd = csv.reader(fd, delimiter="\t", quotechar='"')
            for row in rd:
                if row :
                    tsv = row
        text = ' '.join([tsv[1],tsv[2]]) #get intro and plot of each tsv file
        tsvs[i] = tsv
        cleared = clean(text)

        docwords['document_'+str(i)] = cleared
        allwords += cleared
        
        
    allwords = list(set(allwords)) # get the list of unique words
        for i in range(len(allwords)):
            vocabulary[str(i)] = allwords[i]
            
            
            
    savetojson(path+"\\tsvs.json", tsvs)
    savetojson(path + "\\WORDS\\DocWords.json", docwords)
    savetojson(path + "\\WORDS\\vocabulary.json", vocabulary)

#### 2.1.2) execute query

### 2.2) Conjunctive query & Ranking score

#### 2.2.1) Inverted index

#### 2.2.2)Execute the query

## Defining a new score

## Bonus